<a href="https://colab.research.google.com/github/James-H05/SSI_2021/blob/main/ee_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import earth engine library - should be installed already in colab
import ee

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=HP9j_UjB5S8RBbYeNjaPo9Vb_OuVB9pHI_-QD9PLeTM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWggBQHkHMK1IFNZAM81k8NepgZLUs1AOgXQIS9FUxtcgaa1nAWvqIU

Successfully saved authorization token.


In [6]:
years = ['1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
layers = ['aet', 'def', 'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs']
countries = ['AT', 'BE', 'BG', 'CH', 'CY','CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'EL', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'NL', 'NO', 'PL', 'PT', 'RO', 'SI', 'SK', 'SE', 'UK']

csv = {}

for country in countries:
  for layer in layers:
    csv[country + '_' + layer] = []

for year_index in range(len(years)):
  for month_index in range(len(months)):
    for country in countries:

      if(not (year_index == len(years) and month_index == len(months))):
        start_date = years[year_index] + '-' + months[month_index] + '-01'
        if(months[month_index] != '12'):
          end_date = years[year_index] + '-' + months[month_index+1] + '-01'
        else: # if december
          end_date = years[year_index+1] + '-' + months[0] + '-01'

        #TODO make sure it doesn't crash on the very last month

        terraclimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").filter(ee.Filter.date(start_date, end_date)).first()
        borders = ee.Feature(ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_CODE', country)).first())

        # Reduce the region. The region parameter is the Feature geometry.
        meanDictionary = terraclimate.reduceRegion(
          ee.Reducer.mean(),
          geometry=borders.geometry(),
          scale=30,
          maxPixels=100000000,
          bestEffort=True
        )


        # The result is a Dictionary.  Print it.
        #print(country, start_date, ":", meanDictionary.getInfo())
        meanDictionary.getInfo()
        
        for layer in layers:
          csv[country + '_' + layer].append(meanDictionary.getInfo()[layer])

EEException: ignored